In [1]:
#Run parameters
#used to control every run. Can be user to perfom sensitivity checks
path_dir = r"/Users/../Volumes/Raw/"

low_success = 0.5 #below the median: unsuccessful
high_success = 0.9 #top 10% creators with more followers are deemed successful

low_user_outdegree = 0.25 
high_user_outdegree = 0.75
low_user_activity = 0.25 
high_user_activity = 0.75 

activity_filter = 0
days_delta = 7

In [2]:
import sys  
import pickle
sys.path.insert(0, '/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/')
import numpy as np
import src.utils
from collections import Counter
from src.utils import import_dta, import_tracks_dta,\
gen_active_relations, get_fan_interactions_per_week, calculate_avg_monthly_valence,\
gen_active_relations_prob, get_fan_interactions_per_week_prob, stripplot_prob,\
reaction_probability, follower_list, filter_quantile, sample_creators_music,\
gen_outbound_creators
import numpy as np
import datetime
import pandas as pd
from tqdm import tqdm
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy
import os
from statsmodels.stats.proportion import proportions_ztest
from datetime import datetime, timedelta


In [3]:
def process_date(date):
    '''convert date format like '2013-w09' to '2013-03-04', i.e. the first day of that week'''
    year = date[0:4]
    week = date[6:]
    day = "1"
    date = "{}-{}-1".format(year, week)
    dt = datetime.strptime(date, "%Y-%W-%w")
    return dt

# Data Imports

We start by importing the raw data.  `follows_sent`, `comments_sent`, `shares_sent`, `likes_sent` and `messages_sent` contains data pn the promotional activities that the 35k users tracked in the dataset directed to other users. It includes the `user_id`, the `fan_id` and the `date_sent` which identifies the date when the prom. activity was sent. `users_info_1st` shows the type of user (creator or non-creator, which is identified by a blank) and the date the user entered the platform, for every user that sent or received prom. activities from any of the 35k users tracked in this dataset, while `users_info` contains the same information, but pertaining to the 35k users themselves.

`follows_received` contains information on the follows received by the 35k users and will be used to generate the successful/unsuccessful groups of content creators.

In [4]:
#affiliations :follows
#favoritings :likes

#used in filtering:
path_dir = r"/Users/../Volumes/Raw/"
tracks = import_tracks_dta(path_dir, "12sample_tracks.dta");

#these are the actions sent to 
follows_sent = import_dta(path_dir, "12sample_affiliations_sent.dta");
comments_sent = import_dta(path_dir, "12sample_comments_made.dta");
shares_sent = import_dta(path_dir, "12sample_reposts_made.dta");
likes_sent = import_dta(path_dir, "12sample_favoritings_made.dta");
messages_sent = import_dta(path_dir, "12sample_messages_sent.dta");

#Used to track information on the 1st degree connections
user_info_1st = import_dta(path_dir, "12sample_1st_deg_user_infos.dta");
user_info_1st.columns = ['user_id', 'type', 'entered_platform'];
user_info = import_dta(path_dir, "12sample_user_infos.dta");

#Used to compute creator's success measure
follows_received = import_dta(path_dir, "12sample_affiliations_received.dta");

%%%%%%%%%% 12sample_tracks.dta %%%%%%%%%%
(56262, 7)
%%%%%%%%%% 12sample_affiliations_sent.dta %%%%%%%%%%
(800913, 3)
%%%%%%%%%% 12sample_comments_made.dta %%%%%%%%%%
(29258, 4)
%%%%%%%%%% 12sample_reposts_made.dta %%%%%%%%%%
(179329, 4)
%%%%%%%%%% 12sample_favoritings_made.dta %%%%%%%%%%
(527701, 4)
%%%%%%%%%% 12sample_messages_sent.dta %%%%%%%%%%
(11091, 3)
%%%%%%%%%% 12sample_1st_deg_user_infos.dta %%%%%%%%%%
(670746, 3)
%%%%%%%%%% 12sample_user_infos.dta %%%%%%%%%%
(35000, 3)
%%%%%%%%%% 12sample_affiliations_received.dta %%%%%%%%%%
(432503, 3)


Indegree and outdegree information.

The function below import the outdegree dataset. Because the raw version of those dataset are too large to be processed in memory, we preprocessed them in a separate script.

In [5]:
# Aggregates preprocessed outdegree of 1st degree users
def import_outdegree(path='/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/'):
    d = {}
    for i in range(6):
       d[str(i)] = pd.read_pickle(os.path.join(path,'{}.pkl'.format(i))) 
       d[str(i)]['created_at'] =  pd.to_datetime(d[str(i)]['created_at'])
       d[str(i)]['created_at'] = pd.to_datetime(d[str(i)]['created_at']).dt.floor('d')
       d[str(i)] = d[str(i)].groupby(['sender_id', 'created_at'], as_index = False).size() 
    
    data_outdegree = pd.concat([d['0'], d['1'], d['2'], d['3'], d['4'], d['5']])
    #data_outdegree.set_index('created_at', inplace = True)
    return data_outdegree

In [ ]:
data_outdegree = import_outdegree()

In [ ]:
#data_outdegree = data_outdegree.groupby(['sender_id','created_at'], as_index = False).size()

# Preprocessing

## Creator ids, successful and unsucessful creators

Next, we define three lists of ids: one with the ids from the content creators, according to the `users_info` table, one with the ids of successful creators and the last one with the ids of the unsuccessful ones.

Let's start with a list of the id of creators. We also create a dataset with containing information on creators only.

In [ ]:
mask = (tracks.track_available == 1) & (tracks.public == 't')
creator_ids = tracks[mask].user_id.unique()

creators = tracks[(tracks.track_available == 1) & (tracks.public == 't')]

## Putting together a dataset with the promotional activities made by content creators.

The function `gen_actions_sent_df` creates a dataframe with all the promotional activities that content creators sent to users.

In [ ]:
def gen_actions_sent_df(follows_sent, shares_sent, likes_sent, comments_sent, messages_sent, creator_ids = creator_ids):
    '''
    Creates dataframe containing the actions that content creators send to users.
        Attributes:
                    follows_sent:  dataframe with the follows sent by the 35k users.
                    shares_sent:   dataframe with the shares sent by the 35k users.
                    likes_sent:    dataframe with the likes sent by the 35k users.
                    comments_sent: dataframe with the comments sent by the 35k users.
                    messages_sent: dataframe with the messages sent by the 35k users.
                    creator_ids:   list with content creator ids. If not none, is used to
                                   filter out activities from non creators.
    '''
    
    follows_sent['outbound_activity'] = 'follow'
    follows_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    if 'song_id' in shares_sent.columns:
        shares_sent.drop(columns=["song_id"])
    shares_sent = shares_sent[['reposter_id', "owner_id", 'created_at']]
    shares_sent['outbound_activity'] = 'share'
    shares_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    if 'track_id' in likes_sent.columns:
        likes_sent.drop(columns=["track_id"], inplace=True)
    likes_sent['outbound_activity'] = 'like'
    likes_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    if 'track_id' in comments_sent.columns:
        comments_sent.drop(columns=["track_id"], inplace=True)
    comments_sent['outbound_activity'] = 'comment'
    comments_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    messages_sent["outbound_activity"] = 'message'
    messages_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']
    df = pd.concat([follows_sent, shares_sent, likes_sent, comments_sent, messages_sent])


    if type(creator_ids) == numpy.ndarray:
        df = df[df['user_id'].isin(creator_ids)]
        
    df['week_yr'] = df.date_sent.dt.strftime('%Y-w%U')
    df = df.loc[df['user_id'] != df['fan_id'],:]

    return df

In [ ]:
actions_sent = gen_actions_sent_df(follows_sent, shares_sent, likes_sent, comments_sent,
                                     messages_sent, creator_ids = None)
actions_sent = actions_sent.loc[actions_sent.user_id.isin(creators.user_id.unique())]

In [ ]:
active_users_ids = actions_sent.groupby('user_id', as_index = False).size()
mask = active_users_ids['size']>= activity_filter
active_users_ids = active_users_ids[mask].user_id.unique()

In [ ]:
def successful_creators_followers(follows_received, base_date = datetime(2016, 5, 30, 0, 0), perc1 = None, perc2 = None, subset_creators = None):
    '''Classifies content creators in successful or unsuccessfull
        Arguments:
                    follows_received: dataframe containing the follows received by content creators
                    base date:        date, in datetime.datetime(YYYY, M, DD, H, M) format, in which the number 
                                      of followers per creator is calculated.
                    perc1:            the threshold used to classify unsuccessful content creators. Creator having 
                                      total followers below the number dictated by this threshold, at the base date,
                                      are classified as unsuccessful 
                    perc2:            the threshold used to classify successful content creators. Creator having 
                                      total followers above the number dictated by this threshold, at the base date,
                                      are classified as successful
                    subset_creators:  a pd.DataFrame containing the creators. If is it available, it will be used to 
                                      filter out non creators and to make sure creators with 0 followers are part of
                                      the resulting dataset.
        
    '''
    print(base_date)

    if 'inbound_activity' not in follows_received.columns:
        follows_received.columns = ['fan_id', 'user_id', 'date_sent']

    mask = (follows_received['date_sent'] < base_date)

    df = follows_received[mask].groupby('user_id', as_index=False).agg({'fan_id': pd.Series.nunique})
    df.columns = ['user_id', 'followers']

    
    if type(subset_creators) == pd.DataFrame:
        print('subsetting...')
        df.set_index('user_id', inplace = True)
        df = df.reindex(subset_creators.user_id.unique())
        df.fillna(0, inplace = True)
        df.reset_index(inplace = True)
        df.columns = ['user_id', 'followers']
        
    mask = df.user_id.isin(active_users_ids)
    df = df[mask]

    low = np.quantile(df.followers, perc1)
    high = np.quantile(df.followers, perc2)

    print("High influencer boundary: {}".format(high))
    print("Low influencer boundary: {}".format(low))

    mask = (df["followers"] <= low) | (df["followers"] >= high)
    
    unsuccessful_creator_ids = df.loc[df["followers"] <= low].user_id.unique()
    successful_creator_ids = df.loc[df["followers"] >= high].user_id.unique()

    return unsuccessful_creator_ids, successful_creator_ids


In [ ]:
unsuccessful_ids, successful_ids = successful_creators_followers(follows_received, 
                                                        perc1 = low_success, perc2 = high_success, subset_creators = creators)


In [ ]:
print(len(unsuccessful_ids))
print(len(successful_ids))

In [ ]:
creators.user_id.nunique()

## Filter only actions that were sent to non-fans

We merge the `actions_sent` dataset with a table containing the date each fan started following the creator.

In [ ]:
follows_received.columns = ['fan_id', 'user_id', 'date_sent']
followers = follows_received[["fan_id", "user_id", "date_sent"]]
followers.columns = ["fan_id", "user_id", "follower_since"]

actions_sent = actions_sent.merge(followers, right_on = ['user_id', 'fan_id'],
                                      left_on = ['user_id', 'fan_id'], how = 'left')

Since we are interested in acquisition campaings, we need to produce a dataset that exclude actions targetting fans.
We do that using filters based on the date of the action and the date that the user became a fan of the content creator. The resulting dataframe is named `actions_sent_non_fans`.

We then filter only actions that happened before the user follows the content creator.

In [ ]:
mask = (actions_sent.date_sent < actions_sent.follower_since) | (actions_sent.follower_since.isnull())
actions_sent_non_fans =  actions_sent[mask]
actions_sent_non_fans['week_yr_date'] = actions_sent_non_fans.week_yr.apply(lambda x: process_date(x))

In [ ]:
path_dir_2 = r'/Users/../Volumes/Alter_outbound_activities/'

comments_received_c = import_dta(path_dir_2, "12sample_1st_deg_comments_made.dta");
shares_received_c = import_dta(path_dir_2, "12sample_1st_deg_reposts_made.dta");
likes_received_c = import_dta(path_dir_2, "12sample_1st_deg_favoritings_made.dta");
messages_received_c = import_dta(path_dir_2, "12sample_1st_deg_messages_sent.dta");

In [ ]:
if 'song_id' in shares_received_c:
        shares_received_c.drop(columns=["song_id"])
shares_received_c = shares_received_c[['reposter_id', "owner_id", 'created_at']]
shares_received_c['inbound_activity'] = 'share'
shares_received_c.columns = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

if 'track_id' in likes_received_c:
        likes_received_c = likes_received_c.drop(columns=["track_id"])
likes_received_c['inbound_activity'] = 'like'
likes_received_c.columns = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

if 'track_id' in comments_received_c:
        comments_received_c = comments_received_c.drop(columns=["track_id"])
comments_received_c['inbound_activity'] = 'comment'
comments_received_c.columns_c = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

messages_received_c["outbound_activity"] = 'message'
messages_received_c.columns = ['user_id', 'fan_id', 'date_sent', 'inbound_activity']

In [ ]:
user_activity_data_c = pd.concat([shares_received_c, likes_received_c, comments_received_c, messages_received_c])

# Priori response probabilities

In [ ]:
table_data = pd.read_csv('user_types_ids.csv')

hermit_ids = table_data.loc[table_data.Type =='Hermit'].user_id.unique()
w_a_ids = table_data.loc[table_data.Type =='w_a'].user_id.unique()
f_a_ids = table_data.loc[table_data.Type =='f_a'].user_id.unique()
observer_ids = table_data.loc[table_data.Type =='Observer'].user_id.unique()

In [ ]:
follow_follower_lists = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/Data/alter_follower_lists_3.pkl')

def compute_indirect_return_v3(follower_lists, follows_received_df, indirect_return_df): 
    
    follows_received_df['date_sent'] = follows_received_df['date_sent'].dt.normalize()
    follows_received_df = follows_received_df.loc[(follows_received_df.user_id.isin(creator_ids))]
    follows_received_df = follows_received_df.loc[follows_received_df.fan_id.isin(follower_lists.index.unique())]
    
    for user_id in tqdm(follows_received_df.user_id.unique()):
        df_j = follows_received_df.loc[follows_received_df.user_id == user_id]
        
        for fan_id in follows_received_df.loc[follows_received_df.user_id == user_id].fan_id.unique():
            
            fan_followers = follower_lists.loc[follower_lists.index == fan_id].values[0]
            
            start_date = df_j.loc[(df_j.user_id == user_id)&
                                              (df_j.fan_id == fan_id)].date_sent.iloc[0]
            end_date = datetime(2015, 7, 30)
    
            creator_followers_from_t = df_j.loc[df_j.date_sent >= start_date]\
                                       .groupby('user_id', as_index = False)['fan_id'].apply(np.array)[0]
                
            indirect_return_df.loc[(indirect_return_df['user_id'] == user_id)
                                   & (indirect_return_df['fan_id'] == fan_id), 'Follower_prop'] =\
            sum(np.in1d(creator_followers_from_t, fan_followers))/len(fan_followers)

In [ ]:
indirect_return_df = follows_received.loc[(follows_received.user_id.isin(creator_ids))&
                                          (follows_received.fan_id.isin(follow_follower_lists.index.unique()))
                                          ,['user_id', 'fan_id']].drop_duplicates(subset=['user_id', 'fan_id'])

teste = compute_indirect_return_v3(follow_follower_lists, follows_received, indirect_return_df)

In [ ]:
indirect_return_df

indirect_return_df['user_type'] = indirect_return_df.fan_id.apply(lambda x: 'f_a' if x in f_a_ids else 
                          ('Hermit' if x in hermit_ids else
                          ('Observer' if x in observer_ids else
                          ('w_a' if x in w_a_ids else 'other'))))

## classify content creators
indirect_return_df['creator_type'] = indirect_return_df.user_id.apply(
                               lambda x: 'successful' if x in successful_ids else 
                               ('unsuccessful' if x in unsuccessful_ids else 'other'))

In [ ]:
#indirect_return_df.to_csv('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/Data/all_creators_follower_prop_indegree.csv')

In [ ]:
indirect_return_df.groupby(['user_type'#, 'creator_type'
                           ]).agg({'Follower_prop': ['mean', 'std']})

In [ ]:
indirect_return_df.groupby('user_type').size()

In [ ]:
comments_received_35k = import_dta(path_dir, "12sample_comments_received.dta");
shares_received_35k = import_dta(path_dir, "12sample_reposts_received.dta");
likes_received_35k = import_dta(path_dir, "12sample_favoritings_received.dta");
messages_received_35k = import_dta(path_dir, "12sample_messages_received.dta");

if 'song_id' in shares_received_35k:
        shares_received_35k.drop(columns=["song_id"])
shares_received_35k = shares_received_35k[['reposter_id', "owner_id", 'created_at']]
shares_received_35k['outbound_activity'] = 'share'
shares_received_35k.columns = ['fan_id', 'user_id', 'date_sent', 'outbound_activity']

if 'track_id' in likes_received_35k:
        likes_received_35k = likes_received_35k.drop(columns=["track_id"])
likes_received_35k['outbound_activity'] = 'like'
likes_received_35k.columns = ['fan_id', 'user_id', 'date_sent', 'outbound_activity']

if 'track_id' in comments_received_35k:
        comments_received_35k = comments_received_35k.drop(columns=["track_id"])
comments_received_35k['outbound_activity'] = 'comment'
comments_received_35k.columns = ['fan_id', 'user_id', 'date_sent', 'outbound_activity']

messages_received_35k["outbound_activity"] = 'message'
messages_received_35k.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

user_activity_data_35k = pd.concat([shares_received_35k, likes_received_35k, comments_received_35k, messages_received_35k])

user_activity_data_35k['day_yr_date'] = user_activity_data_35k.date_sent.dt.normalize()

In [ ]:
shares = user_activity_data_35k.loc[user_activity_data_35k.outbound_activity == 'share']

In [ ]:
follows_received.columns = ['fan_id', 'user_id', 'follower_since']

In [ ]:
shares_df = shares.merge(follows_received, on = ['user_id', 'fan_id'], how = 'left')
shares_df = shares_df.loc[shares_df.follower_since.isna() == False]

shares_df.sort_values('date_sent', inplace = True)
shares_df.drop_duplicates(subset=['user_id', 'fan_id'], keep='first', inplace = True)

In [ ]:
follow_follower_lists = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/Data/alter_follower_lists_3.pkl')

def compute_indirect_return_v4(follower_lists, follows_received, shares_follow_df, indirect_return_df): 
    
    shares_follow_df['date_sent'] = shares_follow_df['date_sent'].dt.normalize()
    shares_follow_df = shares_follow_df.loc[shares_follow_df.user_id.isin(creator_ids)]
    shares_follow_df = shares_follow_df.loc[shares_follow_df.fan_id.isin(follower_lists.index.unique())]

    follows_received = follows_received.loc[follows_received.user_id.isin(creator_ids)]
    follows_received = follows_received.loc[follows_received.fan_id.isin(follower_lists.index.unique())]
    follows_received['follower_since'] = follows_received['follower_since'].dt.normalize()
    
    for user_id in tqdm(shares_follow_df.user_id.unique()):
        df_j = shares_follow_df.loc[shares_follow_df.user_id == user_id]
        
        for fan_id in shares_follow_df.loc[shares_follow_df.user_id == user_id].fan_id.unique():
            
            fan_followers = follower_lists.loc[follower_lists.index == fan_id].values[0]
            
            start_date = df_j.loc[(df_j.user_id == user_id)&
                                              (df_j.fan_id == fan_id)].date_sent.iloc[0]
            end_date = datetime(2015, 7, 30)
            
            if follows_received.loc[(follows_received.user_id == user_id) &\
                (follows_received.follower_since >= start_date)].shape[0] == 0:
                pass
            else:
                creator_followers_from_t = follows_received.loc[(follows_received.user_id == user_id) &\
                                     (follows_received.follower_since >= start_date)]\
                                       .groupby('user_id', as_index = False)['fan_id'].apply(np.array)[0]
                
                indirect_return_df.loc[(indirect_return_df['user_id'] == user_id)\
                                   & (indirect_return_df['fan_id'] == fan_id), 'Follower_prop'] =\
            sum(np.in1d(creator_followers_from_t, fan_followers))/len(fan_followers)
                
                #indirect_return_df.loc[(indirect_return_df['user_id'] == user_id)
                 #                  & (indirect_return_df['fan_id'] == fan_id), 'Focal_follower_fb'] =\
                #len(fan_followers)

In [ ]:
indirect_return_df2 = shares_df.loc[(shares_df.user_id.isin(creator_ids))&
                                          (shares_df.fan_id.isin(follow_follower_lists.index.unique()))
                                          ,['user_id', 'fan_id']].drop_duplicates(subset=['user_id', 'fan_id'])

compute_indirect_return_v4(follow_follower_lists, follows_received, shares_df, indirect_return_df2)

In [ ]:
indirect_return_df2['user_type'] = indirect_return_df2.fan_id.apply(lambda x: 'f_a' if x in f_a_ids else 
                          ('Hermit' if x in hermit_ids else
                          ('Observer' if x in observer_ids else
                          ('w_a' if x in w_a_ids else 'other'))))

## classify content creators
indirect_return_df2['creator_type'] = indirect_return_df2.user_id.apply(
                               lambda x: 'successful' if x in successful_ids else 
                               ('unsuccessful' if x in unsuccessful_ids else 'other'))

In [ ]:
#indirect_return_df2.to_csv('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/Data/after_repost_follower_prop_all_creators_indegree.csv')


In [ ]:
#p(follower of focal following creator given a repost by focal)
analysis_data = pd.read_csv('indegree_level.csv', index_col = 0)
analysis_data.columns = ['fan_id', 'followers', 'outdegree_level', 'activities_performed',
       'activity_level', 'Type', 'contact_id', 'indegree', 'indegree_level']

indirect_return_df2.groupby(['user_type'
                             #, 'creator_type'
                            ], as_index = False).agg({'Follower_prop': ['mean', 'median', 'std']})
#it matches the excel

What is the difference between v3 and v4?
v3 uses follows, v4 uses shares!

In [ ]:
analysis_data = pd.read_csv('indegree_level.csv', index_col = 0)
analysis_data.columns = ['fan_id', 'followers', 'outdegree_level', 'activities_performed',
       'activity_level', 'Type', 'contact_id', 'indegree', 'indegree_level']
indirect_return_df2_indegree = indirect_return_df2.merge(analysis_data, on = 'fan_id', how = 'left')

In [ ]:
indirect_return_indegree_df = indirect_return_df2_indegree.groupby(['user_type'
                             #, 'indegree_level'
                            ], as_index = False).agg({'Follower_prop': ['mean', 'median', 'std']})

In [ ]:
df = indirect_return_indegree_df

pivoted_df = round(df.pivot(index='user_type', columns= 'indegree_level', values=[( 'Follower_prop',   'mean')]),3)

# Display the pivoted DataFrame
pivoted_df.T

In [ ]:
indirect_return_df2_indegree = indirect_return_df2_indegree.groupby(['user_type'
                             #, 'indegree_level'
                            ], as_index = False).agg({'Follower_prop': ['mean']})

In [ ]:
df = indirect_return_df2_indegree

pivoted_df = round(df.pivot(index='user_type', columns= 'indegree_level', values=[( 'Follower_prop',   'mean')]),3)

# Display the pivoted DataFrame
pivoted_df.T

In [ ]:
indirect_return_df2_indegree.loc[(indirect_return_df2_indegree.user_type == 'f_a') &(indirect_return_df2_indegree.indegree_level == 'high')]